In [119]:
import sys
import os
import pandas as pd
from pathlib import Path


In [120]:
# Añadir el directorio src al path de Python
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


In [121]:
from calculadora_margen.encoder import Encoder
from calculadora_margen.cleaning import Cleaner
from calculadora_margen.cleaning.params import Parameters
from calculadora_margen.cleaning.validador import Validator
from calculadora_margen.cleaning.outliers_manager import OutliersManager

NameError: name 'detectar_outliers_en_costes' is not defined

In [122]:
project_root_path = Path(src_path).parent
data_path = project_root_path / 'data'
raw_path = data_path / 'raw'
clean_path = data_path / 'clean'

ETL master_lotes

In [123]:
master_lotes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [124]:
cleaner = Cleaner(master_lotes)
params = Parameters.master_lotes

master_lotes = (cleaner
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .drop_duplicates()
    .drop_na(subset=['lote_componente'])
    .get_df()
)

In [125]:
# Creamos clave única para poder hacer merge en otros df
encoder = Encoder(master_lotes)
master_lotes = encoder.create_key(col2='lote_componente', new_col_name='clave_unica')

In [126]:
master_lotes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19252 entries, 4 to 19325
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   articulo         19252 non-null  object
 1   lote_proveedor   19252 non-null  object
 2   lote_componente  19252 non-null  object
 3   clave_unica      19252 non-null  object
dtypes: object(4)
memory usage: 752.0+ KB


In [127]:
master_lotes.to_csv(clean_path / 'master_lotes_clean.csv', index=False)

ETL costes

In [128]:
costes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [129]:
cleaner = Cleaner(costes)
params = Parameters.costes

costes = (cleaner
    .drop_na(subset=['PRCMONEDA'])
    .drop_duplicates()
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .fix_numeric_format(params.cols_to_float)
    .drop_duplicates_batch('lote_componente')
    .get_df()
)

In [130]:
validator = Validator(costes)
params = Parameters.costes

costes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

Tamaño inicial del DataFrame: 8000

Validando columna: componente
Filas inválidas encontradas: 1

Validando columna: lote_componente
Filas inválidas encontradas: 21

Tamaño final del DataFrame: 7978


In [131]:
# Ver las filas inválidas para una columna específica
#invalid_rows = validator.get_invalid('lote_interno')
#print(invalid_rows.head(10))

In [132]:
outliers_manager = OutliersManager(costes)

costes = (outliers_manager
    .process_outliers()
    .clean_columns()
    .get_df()
)


=== RESUMEN DE OUTLIERS ===
Outliers detectados inicialmente: 71
Outliers reemplazados por la media: 66
Outliers restantes: 5

Detalle de outliers restantes:
- MAT101: 1 outliers (desviación media: 0.0%)
- MAT183: 3 outliers (desviación media: 0.0%)
- MAT265: 1 outliers (desviación media: 0.0%)


In [133]:
costes.to_csv(clean_path / 'costes_clean.csv', index=False)

In [134]:
costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7978 entries, 0 to 7977
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   coste_componente_unitario  7978 non-null   float64
 1   lote_componente            7978 non-null   object 
 2   componente                 7978 non-null   object 
dtypes: float64(1), object(2)
memory usage: 187.1+ KB


In [135]:
fabricaciones = pd.read_csv(raw_path / 'fabricaciones_2025.csv',  encoding='UTF-8', sep=';', dtype=str)

In [136]:
cleaner = Cleaner(fabricaciones)
params = Parameters.fabricaciones

fabricaciones = (cleaner
    #.drop_na(subset=['PRCMONEDA'])
    .drop_duplicates()
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .fix_numeric_format(params.cols_to_float)
    .get_df()
)

In [137]:
fabricaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19659 entries, 0 to 19658
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   fecha_fabricacion          19659 non-null  object 
 1   articulo                   19659 non-null  object 
 2   lote_articulo              19654 non-null  object 
 3   unidades_fabricadas        19659 non-null  float64
 4   componente                 19659 non-null  object 
 5   lote_componente_proveedor  19588 non-null  object 
 6   consumo_unitario           19659 non-null  float64
 7   consumo_total              19659 non-null  float64
 8   id_orden                   19659 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.3+ MB
